# Data Loading #

In [1]:
import os
import pandas as pd
import re
import numpy as np

### Perform Counts on Data ###

In [13]:
datapath = "./Data/variants/"
regex = re.compile('[^a-zA-Z]')

longest_ref = 0
longest_ref_string = ""
longest_alt = 0
longest_alt_string = ""
most_variants = 0
counter = 1
max_variants = 0
largest_pos = 0
smallest_pos = 99999999999999
total_files = len(os.listdir(datapath))
for filename in os.listdir(datapath):
    # print("Progress: " + str(counter) + "/" + str(total_files))
    df = pd.read_table(os.path.join(datapath, filename))
    num_variants = 0
    curr_node = -1
    num_variants_total = 0
    for node, pos, ref, alt in zip(df['node_id'], df['POS'], df['REF'], df['ALT']):
        if node > curr_node:
            num_variants = 0
            curr_node = node

        if pos > largest_pos:
            largest_pos = pos
        if pos < smallest_pos:
            smallest_pos = pos
        ref = regex.sub('', ref)
        alt = regex.sub('', alt)
        if len(ref) > longest_ref:
            longest_ref = len(ref)
            longest_ref_string = ref
        if len(alt) > longest_alt:
            longest_alt = len(alt)
            longest_alt_string = alt

        num_variants += 1
        num_variants_total += 1
        if num_variants > most_variants:
            most_variants = num_variants
        if num_variants_total > max_variants:
            max_variants = num_variants_total
    counter += 1

print("Longest Ref: " + str(longest_ref))
print(longest_ref_string)
print("Longest Alt: " + str(longest_alt))
print(longest_alt_string)
print("Most Variants: " + str(most_variants))
print("Max Variants: " + str(max_variants))
print(largest_pos)
print(smallest_pos)

Longest Ref: 226
GTCAATCTCCTGGGCTCCTGGGTTCCGTCTGCCCTGTTCATAGAGACAGAATGAGGGCGTCCCCAGGTGGTGAGCTCAGGGACGTGTGTGTGTGTGTGTGTGTGTGTAACCATCAATCTCCTGGGCTCCTGGGTTCCGTCTGCCCTGTTCATAGGGATGGAATGAGGGCGTCCCCAGGTGGTGAGCTCAGGGACGTTGTGTGTGTGTGTGTGTGTGTGTGTAACCA
Longest Alt: 804
AGTGGTACTTGGTGGTGGTGGTGGTGCTTGGTGATGGTGGTACTTGGTGGTGGTGGTGGTACTTGGTGGTGATGGTACTTGGTGGTGGTGGTACTTGGTGGTGGCAGTGGTACTTGGTGGTGGTACTTGGTGGTACTTGGTGGTGGTGGTGGTACTTGGTGGTGGCGGTGGTACTTGGTGGTGCTGGTGGTGCTTGGTGGTGGTGGTGGTACTTGGTGGTGGTGATGGTACTTGGTGGTGGTGGTGGTGGTACTTGGTGGTGGTACTTGGTGGTACTTGGTGGTGGTGGCGGTACTTGGTGGTGGTGGCGGTGGCACTTGGTGGTGGTGGTGCTTGGTGGTGGTGATGGTACTTGGTGGTGGTGATGGTACTTGGTGGTGGTGGTGGTGGTACTTGGTGGTGAGTGGTACTTGGTGGTGGTGGTGGTGCTTGGTGGTGGTGGTACTTGGTGGTGGTGGTGGTACTTGGTGGTGATGGTACTTGGTGGTGGTGGTACTTGGTGGTGGCAGTGGTACTTGGTGGTGGTACTTGGTGGTACTTGGTGGTGGTGGTGGTACTTGGTGGTGGCGGTGGTACTTGGTGGTGCTGGTGGTGCTTGGTGGTGGTGGTGGTACTTGGTGGTGGTGATGGTACTTGGTGGTGGTGGTGGTGGTACTTGGTGGTGGTACTTGGTGGTACTTGGTGGTGGTGGCGGTACTTGGTGGTGGTGGCGGTGGCACTTGGTGGTGGTGGTGCTTGG

### Generate 1-, 2-, and 3- mer combinations ###
The embedding for a variant will be of size 85; the first element will be the position of the variant normalized using pytorch.normalize with default settings, the next 84 will be the number of occurences of each of the k-mer substrings

In [12]:
bases = ['A', 'C', 'T', 'G']
k_mers = ['A', 'C', 'T', 'G']

for base in bases:
    for base2 in bases:
        k_mers.append(base + base2)

for base in bases:
    for base2 in bases:
        for base3 in bases:
            k_mers.append(base + base2 + base3)

print(len(k_mers))
print(k_mers)

84
['A', 'C', 'T', 'G', 'AA', 'AC', 'AT', 'AG', 'CA', 'CC', 'CT', 'CG', 'TA', 'TC', 'TT', 'TG', 'GA', 'GC', 'GT', 'GG', 'AAA', 'AAC', 'AAT', 'AAG', 'ACA', 'ACC', 'ACT', 'ACG', 'ATA', 'ATC', 'ATT', 'ATG', 'AGA', 'AGC', 'AGT', 'AGG', 'CAA', 'CAC', 'CAT', 'CAG', 'CCA', 'CCC', 'CCT', 'CCG', 'CTA', 'CTC', 'CTT', 'CTG', 'CGA', 'CGC', 'CGT', 'CGG', 'TAA', 'TAC', 'TAT', 'TAG', 'TCA', 'TCC', 'TCT', 'TCG', 'TTA', 'TTC', 'TTT', 'TTG', 'TGA', 'TGC', 'TGT', 'TGG', 'GAA', 'GAC', 'GAT', 'GAG', 'GCA', 'GCC', 'GCT', 'GCG', 'GTA', 'GTC', 'GTT', 'GTG', 'GGA', 'GGC', 'GGT', 'GGG']


### Create Dataset for autoencoder ###
Generate an encoding for each variant in each file, creating a dataset for the autoencoder

In [ ]:
datapath = "./Data/variants/"
regex = re.compile('[^a-zA-Z]')
largest_pos = 246571185
encoded_nucleotides = []
encoded_pos = []

counter = 1
total_files = len(os.listdir(datapath))
for filename in os.listdir(datapath):
    print("Progress: " + str(counter) + "/" + str(total_files))

    df = pd.read_table(os.path.join(datapath, filename))
    for pos, alt in zip(df['POS'], df['ALT']):
        encoded_pos.append(pos)
        encoding = []

        alt = regex.sub('', alt)
        for kmer in k_mers:
            encoding.append(alt.count(kmer))
        
        # print(encoding)
        encoded_nucleotides.append(encoding)
        
    counter += 1

encoded_nucleotides = np.array(encoded_nucleotides)
encoded_pos = np.array(encoded_pos)
print(encoded_nucleotides.shape)
print(encoded_pos.shape)

Save pickle

In [19]:
import pickle

with open('./pickles/encoded_nucleotides.pkl', 'wb') as fp:
    pickle.dump(encoded_nucleotides, fp)
    print("Pickle Successful")

with open('./pickles/encoded_pos.pkl', 'wb') as fp:
    pickle.dump(encoded_pos, fp)
    print("Pickle Successful")

Pickle Successful
Pickle Successful


Load Pickle

In [39]:
with open('./pickles/encoded_nucleotides.pkl', 'rb') as fp:
    encoded_nucleotides = pickle.load(fp)
    print("Pickle loaded")

with open('./pickles/encoded_pos.pkl', 'rb') as fp:
    encoded_pos = pickle.load(fp)
    print("Pickle loaded")

Pickle loaded
Pickle loaded


Normalize the pos values

In [40]:
import torch.nn.functional as f
import torch

encoded_pos = torch.from_numpy(encoded_pos).type(torch.FloatTensor).unsqueeze(1)
encoded_pos = f.normalize(encoded_pos)
print(type(encoded_pos))

<class 'torch.Tensor'>


Concat together pos and nucleotides to form embeddings, create dataset from it

In [41]:
# print(type(encoded_pos))
# print(type(encoded_nucleotides))
encoded_nucleotides = torch.from_numpy(encoded_nucleotides).type(torch.FloatTensor)
encoded_entries = torch.cat([encoded_pos, encoded_nucleotides], dim=1)
print(encoded_entries.shape)

torch.Size([11749258, 85])


Save pickle

In [42]:
import pickle

with open('./pickles/encoded_entries.pkl', 'wb') as fp:
    pickle.dump(encoded_entries, fp)
    print("Pickle Successful")

Pickle Successful


Load Pickkle

In [2]:
import pickle

with open('./pickles/encoded_entries.pkl', 'rb') as fp:
    encoded_entries = pickle.load(fp)
    print("Pickle loaded")

Pickle loaded


## Create VAE to reduce each variant's dimensionality ##

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [18]:
hidden_size = 8

In [19]:
class encoder(nn.Module):
  def __init__(self, in_features, out_features):
    super(encoder, self).__init__()

    self.lin1 = nn.Sequential(
      nn.Linear(in_features, hidden_size)
    )
    self.fc_mu = nn.Linear(hidden_size, out_features)
    self.fc_logvar = nn.Linear(hidden_size, out_features)

  def forward(self, x):
    x = self.lin1(x)
    x_mu = self.fc_mu(x)
    x_logvar = self.fc_logvar(x)

    return x_mu, x_logvar

In [20]:
class decoder(nn.Module):
  def __init__(self, in_features, out_features):
    super(decoder, self).__init__()

    self.lin = nn.Sequential(
      nn.Linear(in_features, hidden_size),
      nn.Linear(hidden_size, out_features)
    )

  def forward(self, x):
    x = self.lin(x)
    return x

In [21]:
class variational_autoencoder(nn.Module): 
  def __init__(self, in_features, latent_dims = 2):
    super(variational_autoencoder,self).__init__()
    #TODO: complete the code to initialize the encoder and the decoder
    # encoder
    self.enc = encoder(in_features, latent_dims)
    # decoder
    self.dec = decoder(latent_dims, in_features)

  def forward(self,x):
    #TODO: complete the forward funciton
    # encoder
    x_mu, x_logvar = self.enc(x)
    # latent z sampling
    latent_z = self.latent_sample(x_mu, x_logvar)
    # decoder
    x = self.dec(latent_z)
    return x, x_mu, x_logvar
  
  def latent_sample(self, mu, logvar):
    #TODO: Sample the latent vector using the reparameterization trick
    # reparameterization, z = mu + std * N(0,I) 
    if self.training:
      # z ~ N(mu, std^2I), note that std should be positive, 
      # logvar can be negative as the output of linear layer, you may consider using exponential function to obtain standard deviation
      std = torch.sqrt(torch.exp(abs(logvar))) #standard deviation
      eps = torch.normal(mu, std) #error term   ##same shape with std, but is ~ N(0,1)
      z = mu + std * eps
      return z
    # evaluation 
    else:
      return mu
  
def MSE_KL(input, target, mu, logvar, beta):
  #TODO: Calculate the loss function 
  # mse is the mean square error between input and output
  loss = nn.MSELoss()
  mse = loss(input, target)
  # kl is the kl-divergence loss
  kl = 1/2 * (-torch.sum(logvar + 1) + torch.sum(logvar) + torch.sum(torch.square(mu)))
  # beta is the regularization value
  return mse + beta * kl

In [23]:
####################Please don't change this cell.##############################

def train_model(model, optimizer, n_epochs=10, beta = 0.001):

    batch_size=256
    losses = []

    # we'll train the network for 10 epochs
    step = 0
    for epoch in range(n_epochs):
        # randomize the order of the data each time through
        random_order = np.random.permutation(encoded_entries.shape[0])
        data_randomized = encoded_entries[random_order]

        # train the network on batches of size `batch_size`
        for data_batch in np.array_split(data_randomized, data_randomized.shape[0] // batch_size):
            step += 1

            # update the network weights to minimize the loss
            output, x_mu, x_logvar = model(data_batch)

            # get loss
            loss = MSE_KL(input = data_batch, target = output, mu = x_mu, logvar = x_logvar, beta = beta)

            # print the loss every 100 epochs
            if step % 100 == 0:
                print("Step: {} Loss: {:.3f}".format(step, loss.item()))

            # backpropagate the loss
            loss.backward()

            # update parameters
            optimizer.step()

            # reset gradients
            optimizer.zero_grad()
            losses.append(loss)

    return losses

####################Please don't change this cell.##############################

In [111]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [24]:
device = "cpu"

In [25]:
# TODO: Please try with different learning rates and find the one that is suitable for our task. You can also change n_epochs or the optimizer

ae = variational_autoencoder(in_features = encoded_entries.shape[1]).to(device)
encoded_entries = encoded_entries.to(device)
# training mode
ae.train()

# Learning rate for the optimizer is a key hyperparameter to play around with when we need to train the model.
learning_rate = .001
optimizer = optim.Adam(ae.parameters(),
                       lr=learning_rate)
losses = train_model(ae, optimizer = optimizer, n_epochs = 10)

Step: 100 Loss: -0.142
Step: 200 Loss: -0.228
Step: 300 Loss: -0.036
Step: 400 Loss: -0.213
Step: 500 Loss: -0.135
Step: 600 Loss: -0.155
Step: 700 Loss: -0.230
Step: 800 Loss: -0.222
Step: 900 Loss: 0.178
Step: 1000 Loss: -0.144
Step: 1100 Loss: -0.065
Step: 1200 Loss: 0.328
Step: 1300 Loss: -0.201
Step: 1400 Loss: -0.212
Step: 1500 Loss: -0.202
Step: 1600 Loss: -0.197
Step: 1700 Loss: -0.203
Step: 1800 Loss: -0.206
Step: 1900 Loss: -0.203
Step: 2000 Loss: -0.111
Step: 2100 Loss: -0.092
Step: 2200 Loss: -0.221
Step: 2300 Loss: -0.189
Step: 2400 Loss: -0.095
Step: 2500 Loss: -0.119
Step: 2600 Loss: -0.156
Step: 2700 Loss: -0.214
Step: 2800 Loss: -0.202
Step: 2900 Loss: -0.151
Step: 3000 Loss: 0.144
Step: 3100 Loss: -0.166
Step: 3200 Loss: 0.401
Step: 3300 Loss: -0.217
Step: 3400 Loss: -0.186
Step: 3500 Loss: -0.185
Step: 3600 Loss: -0.206
Step: 3700 Loss: -0.222
Step: 3800 Loss: -0.073
Step: 3900 Loss: -0.193
Step: 4000 Loss: -0.172
Step: 4100 Loss: -0.179
Step: 4200 Loss: -0.088
Step:

In [ ]:
torch.save(ae.state_dict(), "./pickles/ae")

# Graph Neural Network Classifier #

In [ ]:
import os
import torch
from torchmetrics.classification import MulticlassAUROC
from torch_geometric.loader import DataLoader
from IPython.display import Javascript

In [ ]:
print(torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
batch_size=64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)

In [ ]:
def graph_test(model, loss_fn, loader, device):
     """
     model: pytorch GNN model
     loss_fn: loss function
     loader: DataLoader
     device: device eused to bind the model and tensor
     """
     model.eval()
     auroc_score = 0.0
     ml_auroc = MulticlassAUROC(2, average='macro')
     preds = None
     labels = None

     with torch.no_grad():
          for data in loader:
               out = model(data.x.to(device), data.edge_index.to(device), data.batch.to(device))

               if preds is None:
                    preds = out
               else:
                    preds = torch.cat((preds, out))

               if labels is None:
                    labels = data.y.to(device).long()
               else:
                    labels = torch.cat((labels, data.y.to(device).long()))
     
     auroc_score = ml_auroc(preds, labels) 
     
        
     return auroc_score

In [ ]:
from model.GraphConvNet import GNN

model = GNN()

learning_rate = 1e-3
epoch_num = 200
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

def train():
    model.train()
    total_loss = 0.0
    for data in train_loader:  # Iterate in batches over the training dataset.
        out = model(data.x.to(device), data.edge_index.to(device), data.batch.to(device))  # Perform a single forward pass.
        loss = loss_fn(out, data.y.to(device))  # Compute the loss.
        total_loss += loss
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.
    return total_loss / len(train_loader)

for epoch in range(epoch_num):
    train_loss = train()
    train_auroc = graph_test(model, loss_fn, train_loader, device)
    valid_auroc = graph_test(model, loss_fn, valid_loader, device)
    print(f'Epoch: {epoch:03d}, Train Loss: {train_loss:.5f}, Train Auc: {train_auroc:.4f}, Valid Auc: {valid_auroc:.4f}')